In [3]:
# Aircraft Delay Log - Inference & Evaluation (sistemato)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

# Caricamento dati e modello
df = pd.read_csv('../data/synthetic_logs.csv')
df = df.sample(frac=1.0).reset_index(drop=True)

label_map = {
    0: "TECHNICAL_FAILURE",
    1: "CREW_DELAY",
    2: "WEATHER",
    3: "ATC_RESTRICTION",
    4: "LOGISTICS_ISSUE",
    5: "SECURITY"
}
reverse_label_map = {v: k for k, v in label_map.items()}

# Carica il modello fine-tuned
model = AutoModelForSequenceClassification.from_pretrained("../model/delay_classifier")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model.eval()

# Funzione di predizione
def predict(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1).tolist()
    return [label_map[p] for p in preds]

# Predizione su subset di test
df_test = df[:20].copy()
df_test["predicted"] = predict(df_test["log_text"].tolist())

# Mostra output
print(df_test[["log_text", "label", "predicted"]])

# Valutazione completa
df["label_id"] = df["label"].map(reverse_label_map)
y_true = df["label"].tolist()
y_pred = predict(df["log_text"].tolist())

print("\nClassification Report:")
print(classification_report(y_true, y_pred))


                                             log_text              label  \
0   Holding pattern caused by ATC congestion over ...    ATC_RESTRICTION   
1   Snow accumulation on runway required additiona...            WEATHER   
2   Technical inspection revealed anomaly in fuel ...  TECHNICAL_FAILURE   
3   Extended taxi-in due to temporary runway closu...    ATC_RESTRICTION   
4   Extended taxi-in due to temporary runway closu...    ATC_RESTRICTION   
5   Crew rest requirements caused delay in dispatc...         CREW_DELAY   
6   Missing ground power unit required last-minute...    LOGISTICS_ISSUE   
7   Snow accumulation on runway required additiona...            WEATHER   
8   Crew rest requirements caused delay in dispatc...         CREW_DELAY   
9   Extended taxi-in due to temporary runway closu...    ATC_RESTRICTION   
10  Extended taxi-in due to temporary runway closu...    ATC_RESTRICTION   
11  Missing ground power unit required last-minute...    LOGISTICS_ISSUE   
12  Crew res